# Basic Virtual Worker Tutorial

This notebook contains a basic introduction to PySyft using VirtualWorkers, which are local workers designed to make learning and developement easy (i.e., not require an actual internet connection). All of your workers (performing Torch operations) will live in this one notebook. However, the interfaces will be identical to those that do execute commands over a network.

This tutorial assumes you are already familiar with torch. If you need a primer on PyTorch, check out the following tutorial. (https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)

## Step 1: Initializing a Hook

PySyft, at it's most basic level, is a set of overloaded operations on major deep learning frameworks. (it takes a framework and modifies its default behavior). The object we use to do this is called a Hook, and we must "run a hook" in order to get PySyft's functionality.

In [1]:
import syft as sy
import torch
# this is our hook
hook = sy.TorchHook()

Now all the operations necessary to use PySyft have been overloaded. You can tell that it's been hooked in the following ways.

In [2]:
# don't run this line... just type "torch.na" and then hit the <tab> key to see all the native_ functions
# torch.na

See all the "native" functions that now appear in the package? Those are the original PyTorch functions.. simply renamed old+function-name. All of the actual functions (such as torch.abs()) are actually PySyft code which, if the conditions are right, will call the corresponding "native" functions (such as torch.native_abs()). In fact, the default will do this.

In [3]:
x = sy.FloatTensor([-2,-1,0,1,2,3])
x


-2
-1
 0
 1
 2
 3
[syft.core.frameworks.torch.tensor.FloatTensor of size 6]

In [4]:
x.abs()


 2
 1
 0
 1
 2
 3
[syft.core.frameworks.torch.tensor.FloatTensor of size 6]

You may be wondering... what was the purpose of all this overloading?!?! PyTorch seems no different! Where the functionality for PySyft begins is when we start using what are called Workers. These are machines that can run PyTorch code _remotely_.

## Step 2: Initializing a Remote Worker

As it turns out, you actually already have one worker created (by default) inside of TorchHook. It's called the "local_worker" and it represents the machine you're working on right now (your client).

In [5]:
local = hook.local_worker
local

<syft.core.workers.virtual.VirtualWorker id:0>

Now, you could have initialized your own local worker when you created the TorchHook, but since we didn't do that the hook automatically created one for you of type VirtualWorker. VirtualWorker is a special kind of worker which allows you to _pretend to initialize and talk to another machine_ when in fact all commands are actually running locally. It's very convenient because it allows you to do testing and development in an environment that exactly mimics working with a cluster of remote machines but all of the information is actually running locally (in this notebook!) which makes it easier to introspect (And write unit tests).

In [6]:
remote = sy.VirtualWorker(id=1,hook=hook)

Above we created a new worker with an id of 1.

It's very important that all of your workers have different IDs. In case you were wondering, the local_worker is automatically initialized with an id of 0.

In [7]:
local.id

0

In [8]:
remote.id

1

However, before we start performing remote execution, we need to tell our local worker about our remote worker (so that the local worker... and by extension all of our local PyTorch objects... knows how to communicate with the new remote worker)

In [9]:
local.add_worker(remote)

And done! We are now ready to start performing remote operations.

## Step 3: Sending and Receiving Tensors from Remote Workers

In [10]:
x = sy.FloatTensor([1,2,3,4,5])
x2 = sy.FloatTensor([1,1,1,1,1])

In [11]:
x.send(remote)

FloatTensor[_PointerTensor - id:183194956 owner:0 loc:1 id@loc:34510159450]

In [12]:
x2.send(remote)

FloatTensor[_PointerTensor - id:316535699 owner:0 loc:1 id@loc:63915131380]

Our tensors now live on the remote worker!! We can check in the following way.

In [13]:
x.id

183194956

In [14]:
x2.id

316535699

In [15]:
remote._objects

{34510159450: [_LocalTensor - id:34510159450 owner:1],
 63915131380: [_LocalTensor - id:63915131380 owner:1]}

And there they are!!! Notice that when we print these tensors we can't actually see the data on the client (in this notebook) anymore.

In [16]:
x

FloatTensor[_PointerTensor - id:183194956 owner:0 loc:1 id@loc:34510159450]

In [17]:
print(x)

FloatTensor[_PointerTensor - id:183194956 owner:0 loc:1 id@loc:34510159450]


In [18]:
x2

FloatTensor[_PointerTensor - id:316535699 owner:0 loc:1 id@loc:63915131380]

In [19]:
print(x2)

FloatTensor[_PointerTensor - id:316535699 owner:0 loc:1 id@loc:63915131380]


And if we perform operations... we can't see the result

In [20]:
y = x + x2

In [21]:
y

FloatTensor[_PointerTensor - id:3151760146 owner:0 loc:1 id@loc:4901305215]

In [22]:
print(y)

FloatTensor[_PointerTensor - id:3151760146 owner:0 loc:1 id@loc:4901305215]


But, we can bring all of them back!

In [23]:
x.get()


 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [24]:
x2.get()


 1
 1
 1
 1
 1
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [25]:
y.get()


 2
 3
 4
 5
 6
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]